In [4]:
#load some modules for plotting and math
%matplotlib inline
import matplotlib.pyplot as plt
import numpy
from scipy.integrate import odeint
from ipywidgets import widgets
from IPython.display import display
import unyt as u
plt.style.use('seaborn-poster')

##Biochemical Model Function
def BioChem_FBmodel(y,t):
    x=y[0]
    s=y[1]
    V=y[2]*u.dm**3
    ## Model parameters
    qSmax   = 1.4   /u.hr #maximum growth rate
    Ks      = 0.1         #saturation or Monode constant
    qm      = 0.04  /u.hr #death rate
    Yem     = 0.5   *u.g/u.g  #yield
    Si      = 300         #Substrate initial
    F0      = 0.03  *u.dm**3/u.hr #initial feed rate
    SFR     = 0.2   *u.hr #rate of the exponential feed
    Fmax    = 1.5   *u.dm**3/u.hr #maximum possible feed
    
    ## Exponential Feed Schedule
    F = F0*numpy.exp(SFR*t)     #dm**3/hr
    if F > Fmax:
        F = Fmax
    
    qS   = qSmax*s/(s+Ks) #growth rate of your cells at given substrate concentration (2. Monod eq.) /hr
    My   = (qS-qm)*Yem          #biomass change /hr

    dy = numpy.zeros(3)
    dy[0] = -F/V*x+My*x       
    dy[1] = F/V *(Si-s)-qS*x   # /hr *  g/dm**3" (dimensions "(mass)/(length)**3") and "dimensionless"
    dy[2] = F
    return dy

def f(x_i, s_i, V_i):
    ## Integration Time Scale
    tspan = numpy.linspace(0, 50, 300) *u.hr
    ## Initial Values
    #x_i   = 0.7           #mM
    #s_i   = 0.1           #mM
    #V_i   = 40  *u.dm**3  #L
    y   = [x_i,s_i,V_i]

    ## Call ODE Solver
    sol = odeint(BioChem_FBmodel,y,tspan)
    x, s, v = sol.transpose()
    V=v*u.dm**3  #only to get the units right, does not work with integration of function

    ## Growth and Dillution Rate 
    mu=numpy.gradient(x)/(tspan[1]-tspan[0])/x                          #/hr
    dil=numpy.gradient(V)/(tspan[1]-tspan[0])/V                         #/hr
    X_total=numpy.array([x[i]/V[i] for i in range(x.shape[0])])         #/hr
    deltaDilX_total=numpy.gradient(X_total)/(tspan[1]-tspan[0])/X_total
    print(mu[1])

    ## plotting
    ##simple plot
    #plt.plot(tspan, sol)
    #plt.title('Volume, Substrate and Biomass')
    #plt.xlabel('Time [hr]')
    #plt.ylabel('Volume [liter], Concentration [mM]')
    #plt.show()
    ##plot of Volume, Dilution and Growth Rate
    fig, ax1 = plt.subplots()
    color = 'tab:red'
    ax1.set_xlabel('Time [hr]')
    ax1.set_ylabel('Volume, [liter]', color=color)
    ax1.plot(tspan, V, color=color)
    ax1.tick_params(axis='y', labelcolor=color)
    # instantiate a second axes that shares the same x-axis
    ax2 = ax1.twinx()  
    color = 'tab:blue'
    ax2.set_ylabel('Dilution and Growth Rate $\mu$ [/hr]', color=color) 
    ax2.plot(tspan, mu, color=color, label='growth')
    ax2.plot(tspan, dil, color=color, linestyle='--', label='dilution')
    ax2.plot(tspan, deltaDilX_total, linestyle=':', label='difference')
    ax2.tick_params(axis='y', labelcolor=color)
    handles, labels = ax2.get_legend_handles_labels()
    ax2.legend(handles, labels)
    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    plt.show()
    ##second plot for Biomass and Substrate
    fig, ax1 = plt.subplots()
    color = 'tab:red'
    ax1.set_xlabel('Time [hr]')
    ax1.set_ylabel('Biomass Concentration, [mM/liter]', color=color)
    ax1.plot(tspan, x, color=color)
    ax1.tick_params(axis='y', labelcolor=color)
    # instantiate a second axes that shares the same x-axis
    ax2 = ax1.twinx()  
    color = 'tab:blue'
    ax2.set_ylabel('Substrate Concentration [mM]', color=color) 
    ax2.plot(tspan, s, color=color)
    ax2.tick_params(axis='y', labelcolor=color)
    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    plt.show()

interactive_plot = widgets.interact_manual(f, 
                                           x_i=widgets.FloatSlider(min=0.1, max=2, value=0.7, description='mM'),
                                           s_i=widgets.FloatSlider(min=0.01, max=0.5, value=0.1, description='mM'), 
                                           V_i=widgets.FloatSlider(min=20, max=100, value=40, description='L'))

interactive(children=(FloatSlider(value=0.7, description='mM', max=2.0, min=0.1), FloatSlider(value=0.1, descr…

In [2]:
##load some modules for plotting, math and for the interactive part (slider)
%matplotlib inline
import matplotlib.pyplot as plt
import numpy
from scipy.integrate import odeint
from ipywidgets import widgets
from IPython.display import display
import unyt as u

#Biochemical Model Function
def BioChem_Bmodel(x,t,Ks_d):
    # Model Parameters Values
    MWt_Xyl = 106.2 *u.g/u.mol    # Molecular Weight of Xylene
    Ks_l    = 0.01                # Ks for our cells grown on Xylene (mM)
    max_d   = 0.25  /u.hr         # Maximum Theoretical Deathrate (h-1) 
    max_g   = 0.008 /u.hr         # Maximum Theoretical Growthrate (h-1)
    Yxx     = 2.5                 # Yield of Biomass on Xylene (--)
    #Ks_d    = 58.4                # Ks for death due to Xylene Toxicity (mM)


    # Growth Rate (2. Monod eq.)
    growth = max_g * (x[1]) / (x[1] + Ks_l)
    # Death Rate
    death  = max_d * (x[1]) / (x[1] + Ks_d)

    # Change of Substrate Concentrateion Based on Rate (3. Monod eq.)
    # Use if you are having trouble with substrate < 0 values
    # if x[1] < 0:
    #     rxylene = 0
    # else:
    rxylene = (1/MWt_Xyl) * growth / Yxx

    # Differential for Biomass and Xylene Concentration
    biomass  = (growth - death) * x[0]   # biomass [=] Biomass (mg/L)
    xxylene  = - rxylene * x[0]          # xxylene [=] s, Xylene Concentration (mM)
    return [biomass, xxylene]

##Integration Time Scale
tspan = numpy.linspace(0,340,num=300)



#function to update with call of widget defined below
def f(Ks_d_move, Biomass):
    ##Initial Conditions
    Init_Cond = numpy.array([Biomass,1.0]) #Biomass (mg/L), Xylene Concentration (mM)
    ##Call ODE Solver
    solu = odeint(BioChem_Bmodel,Init_Cond,tspan,args=(Ks_d_move,))
    x_bio, x_xyl = solu.transpose() #Xylene Concentration (mM), Biomass (mg/L)
    ##plotting
    fig, ax1 = plt.subplots(figsize=(14,10))
    color = 'tab:red'
    ax1.set_xlabel('Time [hr]')
    ax1.set_ylabel('Cell Concentration [mg/liter]', color=color)
    ax1.plot(tspan, x_bio, color=color)
    #Analytical solution for unlimited exponential behavior
    initial_growth   =  (x_bio[1]-x_bio[0])/Init_Cond[0]/(tspan[1]-tspan[0])
    plt.ylim(top=x_bio[-1]+10)
    ax1.plot(tspan, Init_Cond[0]*numpy.exp(initial_growth*tspan), color='red', linestyle=':', label='Exponential Growth at Initial Rate')
    handles, labels = ax1.get_legend_handles_labels()
    ax1.legend(handles, labels)
    ax1.tick_params(axis='y', labelcolor=color)
    #instantiate a second axes that shares the same x-axis
    ax2 = ax1.twinx()  
    color = 'tab:blue'
    ax2.set_ylabel('Substrate Concentration [mM]', color=color)
    ax2.plot(tspan, x_xyl, color=color)
    ax2.tick_params(axis='y', labelcolor=color)

interactive_plot = widgets.interact_manual(f, 
                                           Ks_d_move=widgets.FloatSlider(min=20, max=150, value=58.4, description='Xyl. Tox. Lim.'),
                                           Biomass=widgets.FloatSlider(min=20, max=150, value=82.096, description='Biomass'))


interactive(children=(FloatSlider(value=58.4, description='Xyl. Tox. Lim.', max=150.0, min=20.0), FloatSlider(…

In [2]:
import unyt

In [3]:
import matplotlib